In [2]:
import pandas as pd
from konlpy.tag import Mecab
import numpy as np
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# %tensorflow_version 1.x
import tensorflow as tf
from tensorflow import feature_column
import re
from nltk.corpus import stopwords
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
from tensorflow.keras import layers
import pickle
print('TensorFlow Version: {}'.format(tf.__version__))

/home/team/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/team/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/team/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/team/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:5

TensorFlow Version: 1.14.0


In [154]:
news = pd.read_csv("./total_article_ver_half_20200427.csv")

In [6]:
news.columns

Index(['article_category', 'article_title', 'article_content'], dtype='object')

In [155]:
news = news.drop(['article_id', 'article_url', 'article_media', 'article_date'], 1)

In [156]:
news = news[news.article_category=='경제']

In [157]:
photo = news.article_title.str.startswith('포토')

In [158]:
news = news[~photo]

In [159]:
issue = news.article_title.str.startswith('이슈')

In [160]:
news = news[~issue]

In [161]:
news = news.drop(['article_category'], 1)

In [162]:
len(news), len(news.article_content.unique())

(217903, 205076)

In [163]:
news = news.drop_duplicates(['article_content'])

In [164]:
len(news)

205076

In [7]:
def clean_text(text, remove_stopwords = False):
    text = text.lower()
    
    if True:
        text = text.split()
        new_text = []
        for word in text:
            new_text.append(word)
        text = " ".join(new_text)
    
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/’·…]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub(r'［[a-zA-Z가-힣]*］', '', text)
    text = text.strip()
    
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

In [165]:
clean_summaries = []
for summary in news.article_title:
    clean_summaries.append(clean_text(summary))
print("Summaries are complete.")

clean_texts = []
for text in news.article_content:
    clean_texts.append(clean_text(text))
print("Texts are complete.")

Summaries are complete.
Texts are complete.


In [166]:
for i in range(2):
    print("Clean Review #",i+1)
    print(clean_summaries[i])
    print(clean_texts[i])
    print()

Clean Review # 1
carfun ℓ당 100200원 비싼 고급휘발유 돈값 할까
수입차와 함께 급성장하는 고급휘발유시장 고성능차에 주로 쓰는 고급휘발유 국내 수요 6년새 2배 가까이 늘어 출력저하 등 노킹 방지 탁월하지만 국내 휘발유 품질 美 등보다 높아 일반차량은 보통휘발유로도 충분 연비 약간 좋아지는 효과는 있어 현대오일뱅크의 고급휘발유 전용 주유소 ‘카젠  사진제공 현대오일뱅크 서울경제 국내 수입차 이용자가 증가하며 고급휘발유 시장이 급성장하고 있다  애마 愛馬 의 엔진을 보호하고 연비를 높여주는 고급휘발유에 대한 관심이 높아지면서다  정유사들은 2010년 고유가와 함께 시장에서 자취를 감췄던 고급휘발유 마케팅에 다시 불을 붙이고 있다  1일 한국석유공사에 따르면 지난해 고급휘발유 수요는 하루 3 710배럴로 2013년 수요 하루 1 939배럴 의 약 2배 수준으로 늘었다  보통휘발유 수요가 2013년 하루 20만1 275배럴에서 지난해 22만6 948배럴로 약 12 8  증가한 것에 비해 급격한 증가율이다  연간 소비량으로 따져도 고급휘발유는 2016년 88만배럴에서 지난해 135만배럴로 연평균 15 5  늘어난 반면 보통휘발유는 연평균 1 4  증가하는 데 그쳤다  고급휘발유는 고출력 수입차에 주로 사용된다  자동차 연료유 시장에서 고급휘발유 수요의 급성장은 국내 수입차 운전자 수의 증가와 관련이 깊다  한국자동차산업협회에 따르면 국내 수입차 등록대수는 지난 2015년 139만2 035대에서 지난해 241만5 877대로 약 73 6  늘었다  2016년 이후 저유가가 계속되고 있는 점도 고급휘발유 시장 확대에 영향을 미쳤다  업계의 한 관계자는 “최근에는 젊은 층의 수입차 선호 현상이 두드러진다”며 “저유가 지속으로 과거보다는 가격 부담이 덜한 편이라 고급휘발유를 찾는 이들도 늘어나고 있다”고 분석했다  고급휘발유는 고출력 수입차에 보통휘발유를 쓸 때 발생할 수 있는 노킹 현상 방지에 탁월하다  노킹 현상은 휘발유의 이상연소로 엔진룸을 망치로

In [8]:
def count_words(count_dict, text):
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

### 토크나이저_mecab
- 대용량 텍스트 처리 시 mecab이 가장 속도가 빠르지만 window운영체제에서 작동하지 않기 때문에 proto타입 생성 시 Okt를 사용
- 서버에서 본 모형 생성 시 mecab으로 수정

In [53]:
tokenizer = Mecab()

In [168]:
tokens_summaries = [' '.join(tokenizer.morphs(sentence)) for sentence in clean_summaries]

In [170]:
tokens_texts = [' '.join(tokenizer.morphs(sentence)) for sentence in clean_texts]

In [174]:
word_counts = {}

count_words(word_counts, tokens_summaries)
count_words(word_counts, tokens_texts)
            
print("Size of Vocabulary:", len(word_counts))

Size of Vocabulary: 153023


In [176]:
embeddings_index = {}

In [177]:
threshold = 2

In [178]:
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))

Total number of unique words: 153023
Number of words we will use: 115306
Percent of words we will use: 75.35%


In [180]:
embedding_dim = 300
nb_words = len(vocab_to_int)

word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

print(len(word_embedding_matrix))

115306


In [9]:
def convert_to_ints(text, word_count, unk_count, eos=False):
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints)
    return ints, word_count, unk_count

In [182]:
word_count = 0
unk_count = 0

int_summaries, word_count, unk_count = convert_to_ints(tokens_summaries, word_count, unk_count)
int_texts, word_count, unk_count = convert_to_ints(tokens_texts, word_count, unk_count, eos=True)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

Total number of words in headlines: 87091873
Total number of UNKs in headlines: 37721
Percent of words that are UNK: 0.04%


In [10]:
def create_lengths(text):
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

In [183]:
lengths_summaries = create_lengths(int_summaries)
lengths_texts = create_lengths(int_texts)

print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe())

Summaries:
              counts
count  205076.000000
mean       11.951233
std         3.134620
min         1.000000
25%        10.000000
50%        12.000000
75%        14.000000
max        38.000000

Texts:
              counts
count  205076.000000
mean      413.729729
std       311.816657
min         4.000000
25%       214.000000
50%       345.000000
75%       532.000000
max      8025.000000


In [184]:
print(np.percentile(lengths_texts.counts, 90))
print(np.percentile(lengths_texts.counts, 95))
print(np.percentile(lengths_texts.counts, 99))

780.0
957.0
1463.0


In [185]:
print(np.percentile(lengths_summaries.counts, 90))
print(np.percentile(lengths_summaries.counts, 95))
print(np.percentile(lengths_summaries.counts, 99))

16.0
17.0
20.0


In [11]:
def unk_counter(sentence):
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count

In [186]:
sorted_summaries = []
sorted_texts = []
max_text_length = 420
max_summary_length = 20
min_length = 8
unk_text_limit = 0
unk_summary_limit = 0

for length in range(min(lengths_texts.counts), max_text_length): 
    for count, words in enumerate(int_summaries):
        if (len(int_summaries[count]) <= max_summary_length and
            len(int_texts[count]) >= min_length and
            length == len(int_texts[count])
           ):
            sorted_summaries.append(int_summaries[count])
            sorted_texts.append(int_texts[count])
        
print(len(sorted_summaries))
print(len(sorted_texts))

126621
126621


# 로드 시 여기서부터

In [12]:
def model_inputs():
    
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    summary_length = tf.placeholder(tf.int32, (None,), name='summary_length')
    max_summary_length = tf.reduce_max(summary_length, name='max_dec_len')
    text_length = tf.placeholder(tf.int32, (None,), name='text_length')

    return input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length

In [13]:
def process_encoding_input(target_data, vocab_to_int, batch_size):
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [14]:
def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    
    for layer in range(num_layers):
        with tf.variable_scope('encoder_{}'.format(layer)):
            cell_fw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, 
                                                    input_keep_prob = keep_prob)

            cell_bw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, 
                                                    input_keep_prob = keep_prob)

            enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                    cell_bw, 
                                                                    rnn_inputs,
                                                                    sequence_length,
                                                                    dtype=tf.float32)
    enc_output = tf.concat(enc_output,2)
    
    return enc_output, enc_state

In [15]:
def training_decoding_layer(dec_embed_input, summary_length, dec_cell, initial_state, output_layer, 
                            vocab_size, max_summary_length):
    
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=summary_length,
                                                        time_major=False)

    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       initial_state,
                                                       output_layer) 

    training_logits, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                           output_time_major=False,
                                                           impute_finished=True,
                                                           maximum_iterations=max_summary_length)
    return training_logits

In [16]:
def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, initial_state, output_layer,
                             max_summary_length, batch_size):
    
    start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                start_tokens,
                                                                end_token)
                
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        initial_state,
                                                        output_layer)
                
    inference_logits, _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            output_time_major=False,
                                                            impute_finished=True,
                                                            maximum_iterations=max_summary_length)
    
    return inference_logits

In [17]:
def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, text_length, summary_length, 
                   max_summary_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers):
    
    for layer in range(num_layers):
        with tf.variable_scope('decoder_{}'.format(layer)):
            lstm = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, 
                                                     input_keep_prob = keep_prob)
    
    output_layer = Dense(vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
                                                  enc_output,
                                                  text_length,
                                                  normalize=False,
                                                  name='BahdanauAttention')
    
    with tf.name_scope("Attention_Wrapper"):
        
        dec_cell = tf.contrib.seq2seq.AttentionWrapper(dec_cell,
                                                          attn_mech,
                                                          rnn_size)
    initial_state = dec_cell.zero_state(dtype=tf.float32, batch_size=batch_size)
    
    with tf.variable_scope("decode"):
        
        training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                            sequence_length=summary_length,
                                                            time_major=False)
        training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                           training_helper,
                                                           initial_state,
                                                           output_layer) 
        training_logits, _ ,_ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                            output_time_major=False,
                                            impute_finished=True,
                                            maximum_iterations=max_summary_length)
        
    with tf.variable_scope("decode", reuse=True):
        
        start_tokens = tf.tile(tf.constant([vocab_to_int['<GO>']], dtype=tf.int32), [batch_size], name='start_tokens')
        end_token = (tf.constant(vocab_to_int['<EOS>'], dtype=tf.int32))
        inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                    start_tokens,
                                                                    end_token)
        inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                            inference_helper,
                                                            initial_state,
                                                            output_layer)
        inference_logits, _ ,_ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                            output_time_major=False,
                                            impute_finished=True,
                                            maximum_iterations=max_summary_length)

    return training_logits, inference_logits

In [18]:
def seq2seq_model(input_data, target_data, keep_prob, text_length, summary_length, max_summary_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size):
    
    embeddings = word_embedding_matrix
    
    enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)
    enc_output, enc_state = encoding_layer(rnn_size, text_length, num_layers, enc_embed_input, keep_prob)
    
    dec_input = process_encoding_input(target_data, vocab_to_int, batch_size)
    dec_embed_input = tf.nn.embedding_lookup(embeddings, dec_input)
    
    training_logits, inference_logits  = decoding_layer(dec_embed_input, 
                                                        embeddings,
                                                        enc_output,
                                                        enc_state, 
                                                        vocab_size, 
                                                        text_length, 
                                                        summary_length, 
                                                        max_summary_length,
                                                        rnn_size, 
                                                        vocab_to_int, 
                                                        keep_prob, 
                                                        batch_size,
                                                        num_layers)
    
    return training_logits, inference_logits

In [19]:
def pad_sentence_batch(sentence_batch):
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [20]:
def get_batches(summaries, texts, batch_size):
    for batch_i in range(0, len(texts)//batch_size):
        start_i = batch_i * batch_size
        summaries_batch = summaries[start_i:start_i + batch_size]
        texts_batch = texts[start_i:start_i + batch_size]
        pad_summaries_batch = np.array(pad_sentence_batch(summaries_batch))
        pad_texts_batch = np.array(pad_sentence_batch(texts_batch))
        
        pad_summaries_lengths = []
        for summary in pad_summaries_batch:
            pad_summaries_lengths.append(len(summary))
        
        pad_texts_lengths = []
        for text in pad_texts_batch:
            pad_texts_lengths.append(len(text))
        
        yield pad_summaries_batch, pad_texts_batch, pad_summaries_lengths, pad_texts_lengths

In [21]:
epochs = 30
batch_size = 64
rnn_size = 256
num_layers = 3
learning_rate = 0.005
keep_probability = 0.75

In [18]:
with tf.device('/device:XLA_GPU:0'):
    train_graph = tf.Graph()
    with train_graph.as_default():

        input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length = model_inputs()

        training_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                          targets, 
                                                          keep_prob,   
                                                          text_length,
                                                          summary_length,
                                                          max_summary_length,
                                                          len(vocab_to_int)+1,
                                                          rnn_size, 
                                                          num_layers, 
                                                          vocab_to_int,
                                                          batch_size)

        training_logits = tf.identity(training_logits.rnn_output, 'logits')
        inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

        masks = tf.sequence_mask(summary_length, max_summary_length, dtype=tf.float32, name='masks')

        with tf.name_scope("optimization"):
            cost = tf.contrib.seq2seq.sequence_loss(
                training_logits,
                targets,
                masks)

            optimizer = tf.train.AdamOptimizer(learning_rate)

            gradients = optimizer.compute_gradients(cost)
            capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
            train_op = optimizer.apply_gradients(capped_gradients)
    print("Graph is built.")

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argumen

In [20]:
start = 0
end = len(sorted_texts)
sorted_summaries_short = sorted_summaries[start:end]
sorted_texts_short = sorted_texts[start:end]
print("The shortest text length:", len(sorted_texts_short[0]))
print("The longest text length:",len(sorted_texts_short[-1]))

The shortest text length: 11
The longest text length: 419


In [21]:
with tf.device('/device:XLA_GPU:0'):
    learning_rate_decay = 0.95
    min_learning_rate = 0.0005
    display_step = 10
    stop_early = 0 
    stop = 3 
    per_epoch = 3 
    update_check = (len(sorted_texts_short)//batch_size//per_epoch)-1

    update_loss = 0 
    batch_loss = 0
    summary_update_loss = [] 

    checkpoint = "./model_economy_final/real_title.ckpt" 
    with tf.Session(graph=train_graph, config=tf.ConfigProto(log_device_placement=True)) as sess:
    #     writer = tf.summary.FileWriter('./model_IT/graph_proto', train_graph)
        sess.run(tf.global_variables_initializer())
    #     writer.flush()
    
        for epoch_i in range(1, epochs+1):
            update_loss = 0
            batch_loss = 0
            for batch_i, (summaries_batch, texts_batch, summaries_lengths, texts_lengths) in enumerate(
                    get_batches(sorted_summaries_short, sorted_texts_short, batch_size)):
                start_time = time.time()
                _, loss = sess.run(
                    [train_op, cost],
                    {input_data: texts_batch,
                     targets: summaries_batch,
                     lr: learning_rate,
                     summary_length: summaries_lengths,
                     text_length: texts_lengths,
                     keep_prob: keep_probability})

                batch_loss += loss
                update_loss += loss
                end_time = time.time()
                batch_time = end_time - start_time

                if batch_i % display_step == 0 and batch_i > 0:
                    print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                          .format(epoch_i,
                                  epochs, 
                                  batch_i, 
                                  len(sorted_texts_short) // batch_size, 
                                  batch_loss / display_step, 
                                  batch_time*display_step))
                    batch_loss = 0

                if batch_i % update_check == 0 and batch_i > 0:
                    print("Average loss for this update:", round(update_loss/update_check,3))
                    summary_update_loss.append(update_loss)

                    if update_loss <= min(summary_update_loss):
                        print('New Record!') 
                        stop_early = 0
                        saver = tf.train.Saver() 
                        saver.save(sess, checkpoint)

                    else:
                        print("No Improvement.")
                        stop_early += 1
                        if stop_early == stop:
                            break
                    update_loss = 0

            learning_rate *= learning_rate_decay
            if learning_rate < min_learning_rate:
                learning_rate = min_learning_rate

            if stop_early == stop:
                print("Stopping Training.")
                break
    #         writer.close()

Epoch   1/30 Batch   10/1978 - Loss:  8.511, Seconds: 2.29
Epoch   1/30 Batch   20/1978 - Loss:  5.335, Seconds: 2.12
Epoch   1/30 Batch   30/1978 - Loss:  4.753, Seconds: 2.23
Epoch   1/30 Batch   40/1978 - Loss:  4.261, Seconds: 2.38
Epoch   1/30 Batch   50/1978 - Loss:  4.155, Seconds: 2.76
Epoch   1/30 Batch   60/1978 - Loss:  4.082, Seconds: 2.18
Epoch   1/30 Batch   70/1978 - Loss:  4.155, Seconds: 2.25
Epoch   1/30 Batch   80/1978 - Loss:  3.740, Seconds: 2.93
Epoch   1/30 Batch   90/1978 - Loss:  3.624, Seconds: 2.86
Epoch   1/30 Batch  100/1978 - Loss:  3.504, Seconds: 2.42
Epoch   1/30 Batch  110/1978 - Loss:  3.795, Seconds: 2.81
Epoch   1/30 Batch  120/1978 - Loss:  3.891, Seconds: 2.64
Epoch   1/30 Batch  130/1978 - Loss:  3.524, Seconds: 2.76
Epoch   1/30 Batch  140/1978 - Loss:  3.860, Seconds: 2.95
Epoch   1/30 Batch  150/1978 - Loss:  3.649, Seconds: 3.06
Epoch   1/30 Batch  160/1978 - Loss:  3.610, Seconds: 3.32
Epoch   1/30 Batch  170/1978 - Loss:  3.658, Seconds: 2.

Epoch   1/30 Batch 1390/1978 - Loss:  3.035, Seconds: 5.83
Epoch   1/30 Batch 1400/1978 - Loss:  3.104, Seconds: 5.54
Epoch   1/30 Batch 1410/1978 - Loss:  3.089, Seconds: 5.74
Epoch   1/30 Batch 1420/1978 - Loss:  3.123, Seconds: 5.90
Epoch   1/30 Batch 1430/1978 - Loss:  3.244, Seconds: 5.61
Epoch   1/30 Batch 1440/1978 - Loss:  3.070, Seconds: 5.79
Epoch   1/30 Batch 1450/1978 - Loss:  3.172, Seconds: 5.64
Epoch   1/30 Batch 1460/1978 - Loss:  3.187, Seconds: 5.96
Epoch   1/30 Batch 1470/1978 - Loss:  3.128, Seconds: 5.66
Epoch   1/30 Batch 1480/1978 - Loss:  3.093, Seconds: 5.68
Epoch   1/30 Batch 1490/1978 - Loss:  3.253, Seconds: 6.05
Epoch   1/30 Batch 1500/1978 - Loss:  3.086, Seconds: 5.74
Epoch   1/30 Batch 1510/1978 - Loss:  3.132, Seconds: 5.91
Epoch   1/30 Batch 1520/1978 - Loss:  3.123, Seconds: 6.00
Epoch   1/30 Batch 1530/1978 - Loss:  3.023, Seconds: 6.30
Epoch   1/30 Batch 1540/1978 - Loss:  3.112, Seconds: 6.12
Epoch   1/30 Batch 1550/1978 - Loss:  3.227, Seconds: 6.

Epoch   2/30 Batch  800/1978 - Loss:  1.889, Seconds: 4.89
Epoch   2/30 Batch  810/1978 - Loss:  1.940, Seconds: 4.78
Epoch   2/30 Batch  820/1978 - Loss:  1.834, Seconds: 4.90
Epoch   2/30 Batch  830/1978 - Loss:  1.897, Seconds: 4.94
Epoch   2/30 Batch  840/1978 - Loss:  1.953, Seconds: 4.99
Epoch   2/30 Batch  850/1978 - Loss:  1.946, Seconds: 4.97
Epoch   2/30 Batch  860/1978 - Loss:  1.930, Seconds: 5.02
Epoch   2/30 Batch  870/1978 - Loss:  1.923, Seconds: 5.03
Epoch   2/30 Batch  880/1978 - Loss:  1.961, Seconds: 5.03
Epoch   2/30 Batch  890/1978 - Loss:  1.965, Seconds: 5.06
Epoch   2/30 Batch  900/1978 - Loss:  2.020, Seconds: 4.92
Epoch   2/30 Batch  910/1978 - Loss:  1.990, Seconds: 4.80
Epoch   2/30 Batch  920/1978 - Loss:  2.037, Seconds: 4.98
Epoch   2/30 Batch  930/1978 - Loss:  1.987, Seconds: 4.98
Epoch   2/30 Batch  940/1978 - Loss:  2.222, Seconds: 4.86
Epoch   2/30 Batch  950/1978 - Loss:  2.170, Seconds: 4.87
Epoch   2/30 Batch  960/1978 - Loss:  2.182, Seconds: 5.

Epoch   3/30 Batch  210/1978 - Loss:  1.907, Seconds: 3.05
Epoch   3/30 Batch  220/1978 - Loss:  1.739, Seconds: 3.14
Epoch   3/30 Batch  230/1978 - Loss:  1.786, Seconds: 3.51
Epoch   3/30 Batch  240/1978 - Loss:  1.646, Seconds: 3.17
Epoch   3/30 Batch  250/1978 - Loss:  1.642, Seconds: 3.56
Epoch   3/30 Batch  260/1978 - Loss:  1.616, Seconds: 3.59
Epoch   3/30 Batch  270/1978 - Loss:  1.697, Seconds: 3.21
Epoch   3/30 Batch  280/1978 - Loss:  1.726, Seconds: 3.26
Epoch   3/30 Batch  290/1978 - Loss:  1.741, Seconds: 3.43
Epoch   3/30 Batch  300/1978 - Loss:  1.654, Seconds: 3.60
Epoch   3/30 Batch  310/1978 - Loss:  1.589, Seconds: 3.77
Epoch   3/30 Batch  320/1978 - Loss:  1.637, Seconds: 3.88
Epoch   3/30 Batch  330/1978 - Loss:  1.618, Seconds: 3.71
Epoch   3/30 Batch  340/1978 - Loss:  1.583, Seconds: 4.43
Epoch   3/30 Batch  350/1978 - Loss:  1.659, Seconds: 4.06
Epoch   3/30 Batch  360/1978 - Loss:  1.559, Seconds: 3.41
Epoch   3/30 Batch  370/1978 - Loss:  1.643, Seconds: 3.

Epoch   3/30 Batch 1590/1978 - Loss:  2.466, Seconds: 6.24
Epoch   3/30 Batch 1600/1978 - Loss:  2.535, Seconds: 5.95
Epoch   3/30 Batch 1610/1978 - Loss:  2.439, Seconds: 6.13
Epoch   3/30 Batch 1620/1978 - Loss:  2.512, Seconds: 5.95
Epoch   3/30 Batch 1630/1978 - Loss:  2.455, Seconds: 6.51
Epoch   3/30 Batch 1640/1978 - Loss:  2.493, Seconds: 6.38
Epoch   3/30 Batch 1650/1978 - Loss:  2.439, Seconds: 6.18
Epoch   3/30 Batch 1660/1978 - Loss:  2.485, Seconds: 6.55
Epoch   3/30 Batch 1670/1978 - Loss:  2.504, Seconds: 6.24
Epoch   3/30 Batch 1680/1978 - Loss:  2.583, Seconds: 6.09
Epoch   3/30 Batch 1690/1978 - Loss:  2.680, Seconds: 6.00
Epoch   3/30 Batch 1700/1978 - Loss:  2.568, Seconds: 6.26
Epoch   3/30 Batch 1710/1978 - Loss:  2.562, Seconds: 6.48
Epoch   3/30 Batch 1720/1978 - Loss:  2.580, Seconds: 6.68
Epoch   3/30 Batch 1730/1978 - Loss:  2.495, Seconds: 6.73
Epoch   3/30 Batch 1740/1978 - Loss:  2.612, Seconds: 6.61
Epoch   3/30 Batch 1750/1978 - Loss:  2.541, Seconds: 6.

...

Epoch   8/30 Batch   10/1978 - Loss:  1.890, Seconds: 2.29
Epoch   8/30 Batch   20/1978 - Loss:  1.683, Seconds: 2.08
Epoch   8/30 Batch   30/1978 - Loss:  1.751, Seconds: 2.22
Epoch   8/30 Batch   40/1978 - Loss:  1.545, Seconds: 2.39
Epoch   8/30 Batch   50/1978 - Loss:  1.611, Seconds: 2.67
Epoch   8/30 Batch   60/1978 - Loss:  1.592, Seconds: 2.16
Epoch   8/30 Batch   70/1978 - Loss:  1.637, Seconds: 2.45
Epoch   8/30 Batch   80/1978 - Loss:  1.465, Seconds: 2.73
Epoch   8/30 Batch   90/1978 - Loss:  1.474, Seconds: 2.71
Epoch   8/30 Batch  100/1978 - Loss:  1.362, Seconds: 2.51
Epoch   8/30 Batch  110/1978 - Loss:  1.544, Seconds: 2.83
Epoch   8/30 Batch  120/1978 - Loss:  1.581, Seconds: 2.76
Epoch   8/30 Batch  130/1978 - Loss:  1.483, Seconds: 2.76
Epoch   8/30 Batch  140/1978 - Loss:  1.671, Seconds: 2.79
Epoch   8/30 Batch  150/1978 - Loss:  1.615, Seconds: 3.14
Epoch   8/30 Batch  160/1978 - Loss:  1.634, Seconds: 3.25
Epoch   8/30 Batch  170/1978 - Loss:  1.719, Seconds: 2.

Epoch   8/30 Batch 1390/1978 - Loss:  2.025, Seconds: 5.84
Epoch   8/30 Batch 1400/1978 - Loss:  2.070, Seconds: 5.58
Epoch   8/30 Batch 1410/1978 - Loss:  2.054, Seconds: 5.75
Epoch   8/30 Batch 1420/1978 - Loss:  2.049, Seconds: 5.89
Epoch   8/30 Batch 1430/1978 - Loss:  2.195, Seconds: 5.64
Epoch   8/30 Batch 1440/1978 - Loss:  2.068, Seconds: 5.77
Epoch   8/30 Batch 1450/1978 - Loss:  2.108, Seconds: 5.64
Epoch   8/30 Batch 1460/1978 - Loss:  2.137, Seconds: 5.99
Epoch   8/30 Batch 1470/1978 - Loss:  2.089, Seconds: 5.69
Epoch   8/30 Batch 1480/1978 - Loss:  2.098, Seconds: 5.70
Epoch   8/30 Batch 1490/1978 - Loss:  2.186, Seconds: 6.04
Epoch   8/30 Batch 1500/1978 - Loss:  2.114, Seconds: 5.74
Epoch   8/30 Batch 1510/1978 - Loss:  2.130, Seconds: 5.91
Epoch   8/30 Batch 1520/1978 - Loss:  2.147, Seconds: 5.94
Epoch   8/30 Batch 1530/1978 - Loss:  2.069, Seconds: 6.29
Epoch   8/30 Batch 1540/1978 - Loss:  2.100, Seconds: 6.12
Epoch   8/30 Batch 1550/1978 - Loss:  2.216, Seconds: 6.

Epoch   9/30 Batch  800/1978 - Loss:  1.425, Seconds: 4.89
Epoch   9/30 Batch  810/1978 - Loss:  1.499, Seconds: 4.76
Epoch   9/30 Batch  820/1978 - Loss:  1.432, Seconds: 4.92
Epoch   9/30 Batch  830/1978 - Loss:  1.475, Seconds: 4.95
Epoch   9/30 Batch  840/1978 - Loss:  1.498, Seconds: 4.98
Epoch   9/30 Batch  850/1978 - Loss:  1.501, Seconds: 5.01
Epoch   9/30 Batch  860/1978 - Loss:  1.530, Seconds: 5.03
Epoch   9/30 Batch  870/1978 - Loss:  1.508, Seconds: 5.04
Epoch   9/30 Batch  880/1978 - Loss:  1.549, Seconds: 5.05
Epoch   9/30 Batch  890/1978 - Loss:  1.539, Seconds: 5.09
Epoch   9/30 Batch  900/1978 - Loss:  1.565, Seconds: 4.92
Epoch   9/30 Batch  910/1978 - Loss:  1.570, Seconds: 4.79
Epoch   9/30 Batch  920/1978 - Loss:  1.577, Seconds: 4.97
Epoch   9/30 Batch  930/1978 - Loss:  1.550, Seconds: 5.00
Epoch   9/30 Batch  940/1978 - Loss:  1.711, Seconds: 4.86
Epoch   9/30 Batch  950/1978 - Loss:  1.702, Seconds: 4.90
Epoch   9/30 Batch  960/1978 - Loss:  1.702, Seconds: 5.

Epoch  10/30 Batch  210/1978 - Loss:  1.600, Seconds: 3.22
Epoch  10/30 Batch  220/1978 - Loss:  1.455, Seconds: 3.09
Epoch  10/30 Batch  230/1978 - Loss:  1.498, Seconds: 3.51
Epoch  10/30 Batch  240/1978 - Loss:  1.390, Seconds: 3.04
Epoch  10/30 Batch  250/1978 - Loss:  1.409, Seconds: 3.58
Epoch  10/30 Batch  260/1978 - Loss:  1.370, Seconds: 3.60
Epoch  10/30 Batch  270/1978 - Loss:  1.419, Seconds: 2.99
Epoch  10/30 Batch  280/1978 - Loss:  1.457, Seconds: 3.26
Epoch  10/30 Batch  290/1978 - Loss:  1.488, Seconds: 3.43
Epoch  10/30 Batch  300/1978 - Loss:  1.415, Seconds: 3.61
Epoch  10/30 Batch  310/1978 - Loss:  1.337, Seconds: 3.78
Epoch  10/30 Batch  320/1978 - Loss:  1.381, Seconds: 3.81
Epoch  10/30 Batch  330/1978 - Loss:  1.384, Seconds: 3.72
Epoch  10/30 Batch  340/1978 - Loss:  1.361, Seconds: 3.87
Epoch  10/30 Batch  350/1978 - Loss:  1.412, Seconds: 3.94
Epoch  10/30 Batch  360/1978 - Loss:  1.292, Seconds: 3.48
Epoch  10/30 Batch  370/1978 - Loss:  1.365, Seconds: 3.

Epoch  10/30 Batch 1590/1978 - Loss:  2.127, Seconds: 6.26
Epoch  10/30 Batch 1600/1978 - Loss:  2.202, Seconds: 5.91
Epoch  10/30 Batch 1610/1978 - Loss:  2.111, Seconds: 6.10
Epoch  10/30 Batch 1620/1978 - Loss:  2.141, Seconds: 5.99
Epoch  10/30 Batch 1630/1978 - Loss:  2.111, Seconds: 6.49
Epoch  10/30 Batch 1640/1978 - Loss:  2.164, Seconds: 6.34
Epoch  10/30 Batch 1650/1978 - Loss:  2.132, Seconds: 6.23
Epoch  10/30 Batch 1660/1978 - Loss:  2.162, Seconds: 6.56
Epoch  10/30 Batch 1670/1978 - Loss:  2.164, Seconds: 6.24
Epoch  10/30 Batch 1680/1978 - Loss:  2.232, Seconds: 6.26
Epoch  10/30 Batch 1690/1978 - Loss:  2.288, Seconds: 6.12
Epoch  10/30 Batch 1700/1978 - Loss:  2.222, Seconds: 6.31
Epoch  10/30 Batch 1710/1978 - Loss:  2.205, Seconds: 6.49
Epoch  10/30 Batch 1720/1978 - Loss:  2.222, Seconds: 6.69
Epoch  10/30 Batch 1730/1978 - Loss:  2.147, Seconds: 6.72
Epoch  10/30 Batch 1740/1978 - Loss:  2.255, Seconds: 6.57
Epoch  10/30 Batch 1750/1978 - Loss:  2.228, Seconds: 6.

In [5]:
def text_to_seq(text):
    
    text = clean_text(text)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in text.split()]

In [23]:
with tf.device('/device:GPU:0'):
    random = np.random.randint(0,len(tokens_texts))
    input_sentence = tokens_texts[random]
    text = text_to_seq(tokens_texts[random])
    ans = tokens_summaries[random]

    checkpoint = "./model_economy_final/real_title.ckpt"

    loaded_graph = tf.Graph()
    with tf.Session(graph=loaded_graph) as sess:
        loader = tf.train.import_meta_graph(checkpoint + '.meta')
        loader.restore(sess, checkpoint)

        input_data = loaded_graph.get_tensor_by_name('input:0')
        logits = loaded_graph.get_tensor_by_name('predictions:0')
        text_length = loaded_graph.get_tensor_by_name('text_length:0')
        summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
        keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

        answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                          summary_length: [np.random.randint(10, 12)], 
                                          text_length: [len(text)]*batch_size,
                                          keep_prob: 1.0})[0] 

    pad = vocab_to_int["<PAD>"] 

    print('Original Text:', input_sentence)

    print('\nText')
    print('  Word Ids:    {}'.format([i for i in text]))
    print('  Input Words: {}'.format(" ".join([int_to_vocab[i] for i in text])))

    print('\nSummary')
    print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
    print('  Response Words: {}'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))
    print('  원제:', ans)

INFO:tensorflow:Restoring parameters from ./model_economy_final/real_title.ckpt
Original Text: ⓒ 한국거래소 신종 코로나 바이러스 감염증 코로나 19 사태 와 관련 해 금융 위원회 로부터 감사 사업 보 고서 제 출시 한 연장 을 승인 받 은 기업 에 대해 기업지배구조 보고서 공시 시한 이 오 는 7 월 15 일 까지 연장 된다 31 일 한국 거래소 는 기업지배구조 보고서 가이드라인 에서 이 같 은 내용 을 발표 했 다 또한 기존 발표 된 가이드라인 의 10 개 핵심 원칙 을 세분 화 해 이 사회 활동 내역 과 이사 선임 감사 인 에 대한 정보 요구 수준 을 강화 했 는데 이 는 2019 년 도 보고서 에 대한 전수 점검 결과 전반 적 으로 가이드라인 에 따라 충실히 기재 하 고 있 으나 일부 기업 에서 미흡 한 부분 이 발견 됨 에 따라 이 를 수정 보완 한 것 이 다 의무 제출 200 사 자율 제출 9 사 등 총 209 사 중 금융 위원회 지정 양식 보고서 로 공시 한 금융 회사 39 사 를 제외 한 비금 융사 170 사 에 대한 전수 조사 를 실시 했 다 특히 핵심 지표 준수 현황 과 본문 의 내용 이 불 일치 하 는 경우 제도 적 장치 의 도입 여부 만 기재 하 고 상세 한 설명 이 미진 한 경우 가이드라인 에서 기재 대상 의 의미 가 불명확 하 거나 다른 부분 과 중복 등 으로 혼란 이 초래 되 는 경우 를 해소 하 는 방향 으로 개정 됐 다 또한 설문 조사 결과 다수 기업 들 이 작성 사례 를 요청 함 에 따라 세부 원칙 별 모범 사례 및 기피 사례 를 구분 제시 함 으로써 상장 기업 이 올해 보고서 작성 시 참고 할 수 있 도록 했 다는 설명 이 다 거래소 는 가이드라인 을 구성 하 는 10 개 핵심 원칙 에 대해 기존 세부 원칙 23 개 필수 기재사항 30 개 에서 세부 원칙 27 개 필수 기재사항 60 개 로 개정 했 다 또 필수 기재사항 에 구분 번호 를 부여 해 기재 

In [24]:
from rouge import Rouge

In [31]:
rouge = Rouge()
reference = ans
hypothesis = " ".join([int_to_vocab[i] for i in answer_logits if i != pad])

- 'R' : 겹치는 단어 / 원제의 단어
    - R이 1이라면 원제의 모든 단어를 사용하여 제목을 만들었음을 의미.
    - 아무리 긴 문장을 만들어도 단어를 다 사용했다면 1이 나오는 문제 있음.
    
- 'P' : 겹치는 단어 / 요약문의 단어
    - 모델이 생성한 문장 중 실제 제목과 연관있는 단어 비율.
    
- 'ROUGE-1'과 'ROUGE-2'의 차이
    - ROUGE-1은 unigram을 비교
        - 예) the, cat, was, found, under, the, bed
    - ROUGE-2는 bigram을 비교
        - 예) the cat, cat was, was found, found under, under the, the bed
        
- 'ROUGE-L' : 최장 길이로 매칭되는 문자열을 측정
    - ROUGE-2와 달리 연속적 매칭을 요구하지않음
    - 유연한 성능 비교가 가능
    
- 'ROUGE-S' : window size가 주어지면, window size내의 단어쌍들을 묶어서 비교.
    - 예) cat in the hat / window size = 2
        - cat in, cat the, cat hat, in the, in hat, the hat

In [32]:
scores = rouge.get_scores(hypothesis, reference)
scores

[{'rouge-1': {'f': 0.4999999950347222,
   'p': 0.5454545454545454,
   'r': 0.46153846153846156},
  'rouge-2': {'f': 0.36363635867768596, 'p': 0.4, 'r': 0.3333333333333333},
  'rouge-l': {'f': 0.5714285667120181, 'p': 0.75, 'r': 0.46153846153846156}}]